In [19]:
%%writefile app.py

import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN

# ---------------- PAGE CONFIG ----------------
st.set_page_config(
    page_title="Health Analytics Dashboard",
    layout="wide"
)

st.title("🏃 Health Anomaly Detection Dashboard")

# ---------------- FILE UPLOAD ----------------
uploaded_file = st.file_uploader(
    "Upload Fitness Data (CSV)",
    type=["csv"]
)

if uploaded_file:
    df = pd.read_csv(uploaded_file)

    # ---------------- PREPROCESSING ----------------
    df.columns = df.columns.str.lower().str.replace(" ", "_")
    df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")
    df = df.dropna(subset=["timestamp", "user_id"])
    df = df.sort_values(["user_id", "timestamp"])

    for col in df.select_dtypes(include="number"):
        df[col] = df[col].fillna(df[col].mean())

    st.success("✅ Data cleaned successfully")

    # ---------------- SIDEBAR FILTERS ----------------
    st.sidebar.header("🔍 Filters")

    user_id = st.sidebar.selectbox(
        "Select User ID",
        df["user_id"].unique()
    )

    metric = st.sidebar.selectbox(
        "Select Metric",
        ["heart_rate", "sleep", "steps"]
    )

    user_df = df[df["user_id"] == user_id].copy()
    num_records = len(user_df)
    st.write(f"Number of records for selected user: {num_records}")

    # ---------------- DATA CHECK ----------------
    if num_records < 30:
        st.warning(
            "⚠️ Not enough data for fully reliable anomaly detection (less than 30 records)"
        )

    # ---------------- CLUSTERING ----------------
    st.subheader("📊 Activity Clustering")

    cluster_features = user_df[["heart_rate", "steps", "sleep"]].fillna(0)
    X = StandardScaler().fit_transform(cluster_features)
    X_pca = PCA(n_components=2).fit_transform(X)

    labels = DBSCAN(eps=1.3, min_samples=5).fit_predict(X_pca)
    user_df["cluster"] = labels

    fig1, ax1 = plt.subplots()
    ax1.scatter(
        X_pca[:, 0],
        X_pca[:, 1],
        c=labels,
        alpha=0.7
    )
    ax1.set_xlabel("PCA Component 1")
    ax1.set_ylabel("PCA Component 2")
    ax1.set_title("User Activity Clusters")
    st.pyplot(fig1)

    # ---------------- ANOMALY DETECTION ----------------
    st.subheader("🚨 Anomaly Detection")

    anomaly_df = user_df[["timestamp", metric]].dropna().copy()

    # Dynamically adjust rolling window for small datasets
    rolling_window = 5 if num_records >= 5 else max(1, num_records // 2)

    # Smoothing
    anomaly_df["smooth"] = anomaly_df[metric].rolling(
        window=rolling_window, center=True, min_periods=1
    ).mean()

    mean = anomaly_df["smooth"].mean()
    std = anomaly_df["smooth"].std()

    anomaly_df["anomaly"] = abs(anomaly_df["smooth"] - mean) > 2.5 * std

    # Rule-based alerts
    anomaly_df["rule_alert"] = False

    if metric == "heart_rate":
        anomaly_df.loc[
            (anomaly_df[metric] < 50) | (anomaly_df[metric] > 110), "rule_alert"
        ] = True

    if metric == "sleep":
        anomaly_df.loc[
            (anomaly_df[metric] < 4) | (anomaly_df[metric] > 10), "rule_alert"
        ] = True

    if metric == "steps":
        anomaly_df.loc[anomaly_df[metric] < 500, "rule_alert"] = True

    anomaly_df["final_anomaly"] = anomaly_df["anomaly"] | anomaly_df["rule_alert"]

    # ---------------- VISUALIZATION ----------------
    fig2, ax2 = plt.subplots(figsize=(14, 6))
    ax2.plot(
        anomaly_df["timestamp"],
        anomaly_df[metric],
        label="Normal Trend",
        alpha=0.7
    )
    ax2.scatter(
        anomaly_df[anomaly_df["final_anomaly"]]["timestamp"],
        anomaly_df[anomaly_df["final_anomaly"]][metric],
        color="red",
        label="Anomalies"
    )
    ax2.set_title(f"{metric.replace('_',' ').title()} Trend with Anomalies")
    ax2.set_xlabel("Time")
    ax2.set_ylabel(metric.replace("_", " ").title())
    ax2.legend()
    st.pyplot(fig2)

    # ---------------- ANOMALY TABLE ----------------
    st.subheader("📋 Detected Anomalies")
    anomaly_table = anomaly_df[anomaly_df["final_anomaly"]][["timestamp", metric]]
    st.dataframe(anomaly_table)

    # ---------------- DOWNLOAD REPORT ----------------
    csv = anomaly_table.to_csv(index=False).encode("utf-8")
    st.download_button(
        "📥 Download Anomaly Report (CSV)",
        csv,
        file_name="anomaly_report.csv",
        mime="text/csv"
    )

else:
    st.info("⬆️ Upload a CSV file to start analysis")


Overwriting app.py


In [20]:
!pip install streamlit pyngrok prophet scikit-learn matplotlib pandas numpy --quiet
!streamlit run app.py &>/content/streamlit.log &


In [21]:
from pyngrok import ngrok

ngrok.kill()
print("✅ Old ngrok tunnels stopped")


✅ Old ngrok tunnels stopped


In [22]:
from pyngrok import ngrok

ngrok.set_auth_token("36ekI5eRqSoX22mhpXU73je9zDP_2gNgSqZFUqsX8w33iLFjk")

public_url = ngrok.connect(8501)
print("🚀 PUBLIC WEBSITE:", public_url)


🚀 PUBLIC WEBSITE: NgrokTunnel: "https://ominous-ela-overbumptious.ngrok-free.dev" -> "http://localhost:8501"
